In [5]:
import pdfplumber
import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)
text = ""
with pdfplumber.open("Conservatives in Academia.pdf") as pdf:
    for page in pdf.pages:
        text += page.extract_text()



clean_text = text
word_count = len(clean_text.split())
print("word count:",word_count, "\n\n",clean_text)

word count: 1597 

 Volume, Number, Month: Volume I, Number 2, April 2025 [editors will take care of this]
Headline: Affirmative Action, But Make It Conservative
Subhead: An old liberal idea might just be the solution to a new conservative problem.
Tags:
Author(s): Oren Hartstein
Author Position (guest contributor, or staff position): [position]
Author Byline: Mr. Hartstein is a sophomore at Columbia College studying physics and
math. He is a senior editor for Sundial.
Twitter Post (~280 characters before link)
Instagram Post Caption (medium paragraph)
🔗
Full article at the link in bio.
Article Body
REIMP
There is a war between Trump and Columbia—at least, thatʼs what most Columbia
students would like you to believe. They tend to frame the current fight with the Trump
administration as just that—nothing more than a fight with Trump. Itʼs an easy story to
tell, because it portrays such tensions as a one-off political squabble. For many, that
narrative is far preferable to reality: Unive

In [6]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [7]:
from langchain_core.messages import SystemMessage
sys_msg = """You are a helpful assistant that generates article pull-out quotes. 
Each pull out quote should be 30-70 words and should capture the main themes of the article. 
The pull out quotes MUST be direct quotations from the article. 
DO NOT summarize or otherwise modify the direct quotations in any way. A quote can be 1-3 sentences long.

article: \n {article}
"""

In [8]:
from pydantic import BaseModel
from typing import TypedDict

class Quotes(BaseModel):
    quotes: list[str]

class State(TypedDict):
    article: str
    quotes: Quotes


In [9]:
llm = gpt4o_chat.with_structured_output(Quotes)

In [10]:
from langgraph.types import Send
#Node
def quote_generator(state: State):
    return {"quotes": llm.invoke([SystemMessage(sys_msg.format(article=state["article"]))])}

#Conditional Edge
def send_to_validator(state: State):
    return [Send("validator", {"article": state["article"], "quote": q}) for q in state["quotes"]]


In [11]:
from langgraph.graph import START, END, StateGraph
builder = StateGraph(State)

builder.add_node("quote_generator", quote_generator)
builder.add_edge(START, "quote_generator")
builder.add_edge("quote_generator", END)

graph = builder.compile()

config = {"configurable": {"thread_id": "3"}}
result = graph.invoke({"article": clean_text}, config=config)

In [12]:
import re
from typing import List, Dict, Any

def strip_outer_quotes(text: str) -> str:
    if not isinstance(text, str):
        return ""
    return text.strip().strip("\"'“”‘’")

def normalize_text(text: str) -> str:
    """
    Normalizes text by stripping wrapping quotes, unifying punctuation, fixing common
    PDF spacing issues, and standardizing whitespace. Returns lowercase for robust matching.
    """
    if not isinstance(text, str):
        return ""
    # Remove wrapping quotes first
    text = strip_outer_quotes(text)
    # Normalize unicode punctuation and spaces
    replacements = {
        "\u201c": '"', "\u201d": '"',
        "\u2018": "'", "\u2019": "'", "\u02BC": "'",  # apostrophes
        "\u2013": "-", "\u2014": "-", "\u2212": "-", "\u2012": "-", "\u2011": "-",  # dashes
        "\u2026": "...",  # ellipsis
        "\u00A0": " ",    # nbsp
    }
    for src, tgt in replacements.items():
        text = text.replace(src, tgt)
    # Fix missing space after punctuation (common in PDFs)
    text = re.sub(r'([.,!?;:])([A-Za-z])', r'\1 \2', text)
    # Normalize dash spacing
    text = re.sub(r'\s*-\s*', '-', text)
    # Collapse whitespace and lowercase
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

In [13]:
norm_article = normalize_text(clean_text)
quotes_list = result["quotes"].quotes
for quote in quotes_list:
    norm_quote = normalize_text(quote)
    if norm_quote in norm_article:
        continue
    # Very forgiving secondary check (ignore punctuation and spacing)
    stripped_article = re.sub(r'[^a-z0-9]+', ' ', norm_article)
    stripped_quote = re.sub(r'[^a-z0-9]+', ' ', norm_quote)
    if stripped_quote and stripped_quote in stripped_article:
        continue
    # If both checks fail, report an error
    print("ERROR: Quote not in article")
    print(norm_quote)
    print("-"*300)


In [30]:
from PIL import Image, ImageDraw, ImageFont, ImageOps
logo = Image.open("sundial logo.png")

rgb_img = logo.convert('RGB')
alpha_img = logo.split()[-1] # Get the last channel (the alpha mask)
# Invert the RGB part
inverted_rgb = ImageOps.invert(rgb_img)
            
# Recombine the inverted RGB and the original alpha channel
inverted_img = Image.merge('RGBA', (inverted_rgb.split() + (alpha_img,)))

inverted_img        
inverted_img.save("sundial_logo_white.png")
#print(f"Image colors inverted successfully and saved to {output_path}")

In [38]:
from PIL import Image, ImageDraw, ImageFont, ImageOps

def wrap_text(draw, text, font, max_width):
    """
    Wraps text to fit within a specified maximum width.
    This version requires the ImageDraw object to be passed as an argument.
    Returns a list of strings, where each string is a line.
    """
    lines = []
    if not text:
        return lines
    words = text.split()
    current_line = []
    for word in words:
        # Check if adding the next word exceeds max_width
        test_line = " ".join(current_line + [word])
        if draw.textbbox((0, 0), test_line, font=font)[2] <= max_width:
            current_line.append(word)
        else:
            lines.append(" ".join(current_line))
            current_line = [word]
    if current_line:
        lines.append(" ".join(current_line))
    return lines

# 1. Define image dimensions and background color
img_width = 1080  # Standard Instagram post aspect ratio (square)
img_height = 1080
bg_color = (30, 30, 30)  # A dark gray color (R, G, B)

# 2. Create a new image with the specified dimensions and color
image = Image.new('RGB', (img_width, img_height), color=bg_color)

# 3. Get a drawing context
draw = ImageDraw.Draw(image)

# 4. Define the text to display. This is a long paragraph to match the image.
long_text = quotes_list[0]
byline_text = "-Oren Hartstein"
logo_text = "THE FP"

# 5. Load a font (You'll need a font file, e.g., .ttf or .otf)
try:
    font_main = ImageFont.truetype("Times New Roman.ttf", size=50)
    font_byline = ImageFont.truetype("Times New Roman.ttf", size=30)
except IOError:
    print("Font file not found. Using default font.")
    font_main = ImageFont.load_default()
    font_byline = ImageFont.load_default()

# 6. Wrap the main text based on the image width with some padding
padding = 100
wrapped_text = wrap_text(draw, long_text, font_main, img_width - padding * 2)

# 7. Calculate text position to center the entire block
# Sum the height of each line using textbbox()
quote_lines_height = sum(draw.textbbox((0, 0), line, font=font_main)[3] for line in wrapped_text)
total_text_height = quote_lines_height # We'll add the byline and logo separately

start_y = (img_height - total_text_height) / 2

# 8. Draw each line of the wrapped text
text_color = (255, 255, 255)  # A white color for visibility
current_y = start_y

for line in wrapped_text:
    # Use textbbox to get the width of each line for centering
    text_width = draw.textbbox((0, 0), line, font=font_main)[2]
    text_height = draw.textbbox((0, 0), line, font=font_main)[3]
    x = (img_width - text_width) / 2
    draw.text((x, current_y), line, fill=text_color, font=font_main)
    current_y += text_height

# 9. Draw the byline text at the bottom
byline_x = padding
byline_y = img_height - padding - draw.textbbox((0, 0), byline_text, font=font_byline)[3]
byline_color = (255, 100, 100) # Lighter red for contrast
draw.text((byline_x, byline_y), byline_text, fill=byline_color, font=font_byline)

# 10. Load and place the logo image at the bottom-right
# The logo file must be in the same directory as this script
try:
    logo = Image.open("sundial_logo_white.png")
    logo_width = 200 # Set a desired width for the logo
    aspect_ratio = logo.width / logo.height
    logo_height = int(logo_width / aspect_ratio)

    logo = logo.resize((logo_width, logo_height), Image.Resampling.LANCZOS)

    # Position the logo in the bottom right corner with padding
    logo_x = img_width - padding - logo.width + 80
    logo_y = img_height - padding - logo.height + 130

    # Paste the logo onto the main image using its alpha channel for transparency
    image.paste(logo, (logo_x, logo_y), logo)

except FileNotFoundError:
    print("Logo file 'logo.png' not found. Skipping logo placement.")
except Exception as e:
    print(f"An error occurred while processing the logo: {e}")

# 11. Save the image
image.save("insta_post_with_logo.png")
print("Image 'insta_post_with_logo.png' generated successfully!")


Image 'insta_post_with_logo.png' generated successfully!
